In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "CIFAR10",
            # 'dataset': "FMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            'per_task_examples': 10000,
            'per_task_examples': np.inf,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 10,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.002
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=CIFAR10/seed=1_epoch=1_lr=0.001_alpha=0.002_tau=10


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                      per_task_memory_examples=params['per_task_memory_examples'],
                      per_task_examples = min(params['per_task_examples'], 10000),
                      random_class_idx = False)
    label_li = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_feature = 28*28

elif params['dataset'] == 'FMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                             per_task_memory_examples=params['per_task_memory_examples'],
                             per_task_examples = min(params['per_task_examples'], 15000),
                             random_class_idx = False)

    label_li = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 
                  'Ankel boot']
    n_feature = 28*28
    
elif params['dataset'] == 'CIFAR10':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = min(params['per_task_examples'], 15000),
                        random_class_idx = False)
    n_feature = 32*32*3

[0 1 2 3 4 5 6 7 8 9]
Files already downloaded and verified
Files already downloaded and verified


In [3]:
benchmark.trains[1].inputs.shape

torch.Size([10000, 3, 32, 32])

In [4]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2

# backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
backbone = cl.backbones.ResNet18Small(config=params)
algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [5]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
solver=<function LS_solver at 0x7fe36126d940>
[1] Eval metrics for task 1 >> {'accuracy': 87.7, 'loss': 0.0011314899250864984, 'std': 1.7000000000000015}
training_task_end
---------------------------- Task 2 -----------------------
solver=<function LS_solver at 0x7fe36126d940>
losses=tensor([[0.9722, 1.0474, 7.6495, 7.7716]])
A_np.shape=(4, 10000)
b_np.shape=(4,)
Elapsed time:3.069
Loss difference:[-1.24018651e-07 -1.29087846e-07 -1.21286350e-07 -1.26380458e-07]
len(updated_seq_indices)=10000
[2] Eval metrics for task 1 >> {'accuracy': 59.699999999999996, 'loss': 0.028173024654388427, 'std': 0.8000000000000007}
[2] Eval metrics for task 2 >> {'accuracy': 0.0, 'loss': 400000020.48, 'std': 0.0}
training_task_end
---------------------------- Task 3 -----------------------
solver=<function LS_solver at 0x7fe36126d940>
losses=tensor([[2.7716e-01, 1.3845e-02, 3.2855e+00, 4.7619e+00, 4.8234e+01, 7.8249e+01]])
A_np.shape=(6, 10000)
b_

In [6]:
metric_manager_callback.meters['std'].get_std()

[1.7000000000000015, 29.855359652832856, 0.0, 0.0, 30.0]

In [7]:
metric_manager_callback.meters['accuracy'].get_data()

array([[87.7,  0. ,  0. ,  0. ,  0. ],
       [59.7,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ],
       [50. ,  0. ,  0. ,  0. ,  0. ]])

In [8]:
metric_manager_callback.meters['accuracy'].compute_overall()

/home/jaeyoung/anaconda3/envs/cil/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jaeyoung/anaconda3/envs/cil/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[87.7, 59.699999999999996, nan, nan, 50.0]